In [1]:
import tensorflow as tf
from os import path, getcwd, chdir

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [19]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') >= 0.998):
      print("Reached 99.8% accuracy so cancelling training!")
      self.model.stop_training = True

def train_mnist_conv():
  
  # Load the data
  mnist = tf.keras.datasets.mnist
  (training_images, training_labels), (test_images, test_labels) = mnist.load_data()

  # Reshape and then normalize the data
  training_images = training_images.reshape(60000, 28, 28, 1)
  training_images = training_images / 255

  test_images = test_images.reshape(10000, 28, 28, 1)
  test_images = test_images / 255

  # Create the model
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
                                      tf.keras.layers.MaxPooling2D((2, 2)),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(128, activation='relu'),
                                      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.summary()

  # Compile the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Fit the model
  callback = MyCallback()
  history = model.fit(training_images, training_labels, epochs=20, callbacks=[callback])

  # Test the model
  results = model.evaluate(test_images, test_labels)
  print(f"results: {results}")

  return history.epoch, history.history['accuracy'][-1]


In [20]:
train_mnist_conv()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 10816)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               1384576   
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
Total params: 1,386,506
Trainable params: 1,386,506
Non-trainable params: 0
____________________________________________

AttributeError: ignored